<a href="https://colab.research.google.com/github/fedefliguer/trading/blob/master/v1/v1_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

import yfinance as yf
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
pd.options.mode.chained_assignment = None
from datetime import date
from datetime import timedelta 
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import fbeta_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from io import BytesIO
import pickle
import requests

pd.set_option('display.max_columns', 400)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.width', 1000)

def descarga(ticker, fc_empieza, fc_termina):
  base = yf.download(ticker, start=fc_empieza, end=fc_termina)
  base = base[['Close', 'Volume', 'High', 'Low']]
  base.insert(loc=0, column='Ticker', value=ticker)
  base.reset_index(level=0, inplace=True)
  base.columns=['fc', 'ticker', 'y', 'vl', 'high', 'low']
  return base

def calcula_pc_merval(dataset):
  dataset = pd.merge(dataset,mvl,on='fc',how='left')
  dataset['pc_merval'] = dataset.y/dataset.mvl
  dataset = dataset.drop(['mvl'], axis=1)
  return dataset

def calcula_amplitud(dataset):
  dataset['amplitud'] = (dataset.high - dataset.low)/dataset.y
  return dataset

def estandariza_volumen(dataset):
  mean_vl = dataset['vl'].mean()
  std_vl = dataset['vl'].std()
  dataset['vl'] = (dataset.vl - mean_vl)/std_vl
  return dataset

def calcula_medias(dataset):
  period = 12
  sma = dataset['y'].rolling(period, min_periods=period).mean()
  idx_start = sma.isna().sum() + 1 - period
  idx_end = idx_start + period
  sma = sma[idx_start: idx_end]
  rest = dataset['y'][idx_end:]
  ema = pd.concat([sma, rest]).ewm(span=period, adjust=False).mean()
  dataset['exp1'] = ema
  period = 26
  sma = dataset['y'].rolling(period, min_periods=period).mean()
  idx_start = sma.isna().sum() + 1 - period
  idx_end = idx_start + period
  sma = sma[idx_start: idx_end]
  rest = dataset['y'][idx_end:]
  ema = pd.concat([sma, rest]).ewm(span=period, adjust=False).mean()
  dataset['exp2'] = ema
  macd = dataset['exp1']-dataset['exp2']
  dataset['macd'] = macd
  dataset['exp3'] = macd.ewm(span=9, adjust=False).mean()
  dataset['histog'] = dataset['macd'] - dataset['exp3'] 
  return dataset

def calcula_historia(dataset, lags):
  for (columnName, columnData) in dataset.iloc[:,6:].iteritems():
    i = 1
    while i < lags:
      colname = "var_%s_%s" % (columnName, i)
      dataset[colname] = columnData/columnData.shift(i)-1
      i = i + 1
  return dataset

def calcula_canalidad_y(dataset):
  i = 1
  dataset['lag_y_1'] = dataset.y.shift(1)
  dataset['nu_dias_y_entre_max_min_30'] = np.where((dataset['lag_y_1'] < dataset['high']) & (dataset['lag_y_1'] > dataset['low']), 1, 0)
  dataset['nu_dias_y_entre_5pc_30'] = np.where((dataset['lag_y_1'] < (dataset.y * 1.05)) & (dataset['lag_y_1'] > (dataset.y * 0.95)), 1, 0)

  dataset['nu_dias_y_entre_max_min_90'] = np.where((dataset['lag_y_1'] < dataset['high']) & (dataset['lag_y_1'] > dataset['low']), 1, 0)
  dataset['nu_dias_y_entre_5pc_90'] = np.where((dataset['lag_y_1'] < (dataset.y * 1.05)) & (dataset['lag_y_1'] > (dataset.y * 0.95)), 1, 0)

  dataset['nu_dias_y_entre_max_min_180'] = np.where((dataset['lag_y_1'] < dataset['high']) & (dataset['lag_y_1'] > dataset['low']), 1, 0)
  dataset['nu_dias_y_entre_5pc_180'] = np.where((dataset['lag_y_1'] < (dataset.y * 1.05)) & (dataset['lag_y_1'] > (dataset.y * 0.95)), 1, 0)

  dataset = dataset.drop(['lag_y_1'], axis=1)
  i = 2
  while i < 30:
    colname = "lag_y_%s" % (i)
    dataset[colname] = dataset.y.shift(i)
    dataset['nu_dias_y_entre_max_min_30'] = dataset['nu_dias_y_entre_max_min_30'] + np.where((dataset[colname] < dataset['high']) & (dataset[colname] > dataset['low']), 1, 0)
    dataset['nu_dias_y_entre_5pc_30'] = dataset['nu_dias_y_entre_5pc_30'] + np.where((dataset[colname] < (dataset.y * 1.05)) & (dataset[colname] > (dataset.y * 0.95)), 1, 0)
    i = i + 1
    dataset = dataset.drop([colname], axis=1)

  i = 2
  while i < 90:
    colname = "lag_y_%s" % (i)
    dataset[colname] = dataset.y.shift(i)
    dataset['nu_dias_y_entre_max_min_90'] = dataset['nu_dias_y_entre_max_min_90'] + np.where((dataset[colname] < dataset['high']) & (dataset[colname] > dataset['low']), 1, 0)
    dataset['nu_dias_y_entre_5pc_90'] = dataset['nu_dias_y_entre_5pc_90'] + np.where((dataset[colname] < (dataset.y * 1.05)) & (dataset[colname] > (dataset.y * 0.95)), 1, 0)
    i = i + 1
    dataset = dataset.drop([colname], axis=1)

  i = 2
  while i < 180:
    colname = "lag_y_%s" % (i)
    dataset[colname] = dataset.y.shift(i)
    dataset['nu_dias_y_entre_max_min_180'] = dataset['nu_dias_y_entre_max_min_180'] + np.where((dataset[colname] < dataset['high']) & (dataset[colname] > dataset['low']), 1, 0)
    dataset['nu_dias_y_entre_5pc_180'] = dataset['nu_dias_y_entre_5pc_180'] + np.where((dataset[colname] < (dataset.y * 1.05)) & (dataset[colname] > (dataset.y * 0.95)), 1, 0)
    i = i + 1
    dataset = dataset.drop([colname], axis=1)

  return dataset

def calcula_canalidad_histog_macd(dataset):
  list = [5, 30, 90, 180]
  for ventana in list:
    i = 1
    dataset['lag_histog_1'] = dataset.histog.shift(1)
    colname_nu_1 = "nu_dias_histog_entre_5pc_%s" % (ventana)
    dataset[colname_nu_1] = np.where((dataset['lag_histog_1'] < (dataset.histog * 1.05)) & (dataset['lag_histog_1'] > (dataset.histog * 0.95)), 1, 0)

    colname_nu_2 = "nu_dias_histog_positivo_%s" % (ventana)
    dataset[colname_nu_2] = np.where((dataset['lag_histog_1']>0), 1, 0)

    colname_nu_3 = "nu_dias_histog_negativo_%s" % (ventana)
    dataset[colname_nu_3] = np.where((dataset['lag_histog_1']<0), 1, 0)

    colname_nu_4 = "nu_dias_histog_mismo_signo_%s" % (ventana)
    dataset[colname_nu_4] = np.where(((dataset['lag_histog_1']>0) & (dataset['histog']>0))|((dataset['lag_histog_1']<0) & (dataset['histog']<0)), 1, 0)

    dataset = dataset.drop(['lag_histog_1'], axis=1)
    i = 2
    while i < (ventana+1):
      colname = "lag_histog_%s" % (i)
      dataset[colname] = dataset.histog.shift(i)
      dataset[colname_nu_1] = dataset[colname_nu_1] + np.where((dataset[colname] < (dataset.histog * 1.50)) & (dataset[colname] > (dataset.histog * 0.50)), 1, 0)
      dataset[colname_nu_2] = dataset[colname_nu_2] + np.where((dataset[colname]>0), 1, 0)
      dataset[colname_nu_3] = dataset[colname_nu_3] + np.where((dataset[colname]<0), 1, 0)
      dataset[colname_nu_4] = dataset[colname_nu_4] + np.where(((dataset[colname]>0) & (dataset['histog']>0))|((dataset[colname]<0) & (dataset['histog']<0)), 1, 0)
      i = i + 1
      dataset = dataset.drop([colname], axis=1)
  return dataset

def calcula_AT_tendencias(dataset,lags):
  print("Empiezo AT", lags)
  dataset_aux = dataset.copy()

  # Construye las columnas para determinar si es un pico
  i = 1
  while i < (lags+1):
      colname = 'p%sb' % (i)                                                  
      dataset_aux[colname] = round(dataset_aux.y.shift(i),2)
      j = i * -1
      colname = 'p%sf' % (-j)                                                  
      dataset_aux[colname] = round(dataset_aux.y.shift(j),2)
      i = i + 1

  # Determina si es un pico  
  dataset_aux['maxb'] = round(dataset_aux.filter(regex=(".*b")).max(axis=1),2)
  dataset_aux['maxf']= round(dataset_aux.filter(regex=(".*f")).max(axis=1),2)
  dataset_aux['minb'] = round(dataset_aux.filter(regex=(".*b")).min(axis=1),2)
  dataset_aux['minf'] = round(dataset_aux.filter(regex=(".*f")).min(axis=1),2)
  dataset_aux['T'] = np.where((dataset_aux['y']>dataset_aux['maxb']) & (dataset_aux['y']>dataset_aux['maxf']) & ((pd.to_datetime(date.today()) - dataset_aux.fc).dt.days < lags*75), 1, 0) # Le agrego esta condición por performance, solo revisa picos relativos a fechas recientes segun los lagos (lag de 4, 300 días; lag de 8, 600)
  dataset_aux['P'] = np.where((dataset_aux['y']<dataset_aux['minb']) & (dataset_aux['y']<dataset_aux['minf']) & ((pd.to_datetime(date.today()) - dataset_aux.fc).dt.days < lags*75), 1, 0) # 

  techos = dataset_aux[(dataset_aux['T']==1)]
  techos['m'] = (techos.y.shift(1) - techos.y)/(techos.fc.shift(1) - techos.fc).dt.days
  techos.name = 'techos'
  pisos = dataset_aux[(dataset_aux['P']==1)]
  pisos['m'] = (pisos.y.shift(1) - pisos.y)/(pisos.fc.shift(1) - pisos.fc).dt.days
  pisos.name = 'pisos'
  dataset_aux_list = [techos, pisos]
#  print("Detecté picos")

  for dataset_aux_picos in dataset_aux_list:  # En cada dataset_aux (techos y pisos)
#    print("Voy a desarrollar picos")
    name = dataset_aux_picos.name
    dias = len(dataset_aux)
    for index, row in dataset_aux_picos.iloc[1:].iterrows(): # Para cada pico detectado (fila del dataset_aux) a partir del segundo (porque el primero no tiene anterior, no tiene tendencia)
#      print("Voy a desarrollar un pico")
      y_start = row['y']
      pendiente = row['m']
      if (dias < np.where(dataset_aux.fc==row['fc'])[0] + lags):
        continue    
      serie = [] # Crea la serie que va a contener el precio proyectado
      serie = np.append(serie, np.repeat(np.nan, (np.where(dataset_aux.fc==row['fc'])[0] + lags))) # Appendea nulos hasta el día en el que confirmamos que nació una tendencia
      i = np.where(dataset_aux.fc==row['fc'])[0] + lags
      while (i < dias):
        dia = i - (np.where(dataset_aux.fc==row['fc'])[0] + lags)
        serie = np.append(serie, (y_start + pendiente*lags) + pendiente*dia)
        i = i + 1 # Appendea el precio proyectado hasta el final

      colname = '%s_%s_proy' % (name, index)  # Precio proyectado
      dataset_aux[colname] = serie # Construye la columna de toda la serie
#      print("Voy a calcular pruebas y pasajes")

      # Construyo columna con veces en la que el pico fue superado
      colname_pass = '%s_%s_pass' % (name, index) # Pico pasado
      if name == 'techos':
        dataset_aux[colname_pass] = np.where(dataset_aux['y']>(dataset_aux[colname])*1.005, 1, 0)
      elif name == 'pisos':
        dataset_aux[colname_pass] = np.where(dataset_aux['y']<(dataset_aux[colname])*0.995, 1, 0)
      dataset_aux[colname_pass] = dataset_aux[colname_pass].cumsum()

      # Construyo columna con veces en la que el pico fue probado
      colname_prueba = '%s_%s_prueba' % (name, index)  
      dataset_aux[colname_prueba] = np.where((dataset_aux['y']>dataset_aux[colname]*0.995)&(dataset_aux['y']<dataset_aux[colname]*1.005), 1, 0)
      dataset_aux[colname_prueba] = dataset_aux[colname_prueba].cumsum()

      # Construyo columna con pendiente del pico
      colname_pendiente = '%s_%s_pendiente' % (name, index)  
      dataset_aux[colname_pendiente] = row['m']

      # Creo la combinacion y elimino cada uno
      colname_comb = '%s_%s' % (name, index)
      dataset_aux[colname_comb] = dataset_aux[[colname, colname_pass, colname_prueba, colname_pendiente]].values.tolist()
      del dataset_aux[colname]
      del dataset_aux[colname_pass]
      del dataset_aux[colname_prueba]
      del dataset_aux[colname_pendiente]
#      print("Desarrollé todo el pico")

  # Creo el objeto por cada techo o piso individual
  names_techos = dataset_aux.filter(regex=("(techos)(.*)")).columns
  names_pisos = dataset_aux.filter(regex=("(pisos)(.*)")).columns

#  print("Voy a iterar sobre los precios (solo el último)")
  for index, row in dataset_aux.iterrows():  # Por cada fila del dataset_aux original (por cada precio)
    if index != dataset_aux.index[-1]:
#      print("No es el último, salteo")
      continue
    else:
#      print("Es el último, genero variables")

      # Genero las rows vacías con las variables agregadas
      nu_pruebas_techo_vivo_mas_probado = np.nan    
      precio_proyectado_techo_vivo_mas_probado = np.nan
      precio_proyectado_techo_vivo_mas_cercano = np.nan
      precio_proyectado_techo_muerto_mas_cercano = np.nan
      tendencia_techo_vivo_mas_probado = np.nan

      nu_pruebas_piso_vivo_mas_probado = np.nan
      precio_proyectado_piso_vivo_mas_probado = np.nan
      precio_proyectado_piso_vivo_mas_cercano = np.nan
      precio_proyectado_piso_muerto_mas_cercano = np.nan
      tendencia_piso_vivo_mas_probado = np.nan

      # Voy a recorrer cada tendencia proyectada para definir cuáles van, en caso de que corresponda lo asigno a estas variables agregadas

      i = 0
      while i < len(row.index): # Por cada uno de los picos de los que se puede armar tendencia
        if (row.index[i] in names_techos):  # Si es un techo
          if row[i][1]>5: # Si está muerto
            if abs(row['y']-row[i][0]) < abs(row['y']-precio_proyectado_techo_muerto_mas_cercano) or np.isnan(precio_proyectado_techo_muerto_mas_cercano): # Si está muerto y proyecta precio más cercano que el actual
              precio_proyectado_techo_muerto_mas_cercano = row[i][0]
              
          else: # Si está vivo
            if row[i][2] > nu_pruebas_techo_vivo_mas_probado or (np.isnan(nu_pruebas_techo_vivo_mas_probado) and row[i][2]>0): # Si fue más probado que el actual
              nu_pruebas_techo_vivo_mas_probado = row[i][2]
              precio_proyectado_techo_vivo_mas_probado = row[i][0]
              tendencia_techo_vivo_mas_probado = row[i][3]

            if (np.isnan(precio_proyectado_techo_vivo_mas_cercano)) or (abs(row['y']-row[i][0]) < abs(row['y']-precio_proyectado_techo_vivo_mas_cercano)): # Si, sin haber muerto, proyecta un techo más alto que el actual
              precio_proyectado_techo_vivo_mas_cercano = row[i][0]

        elif (row.index[i] in names_pisos):
          if row[i][1]>5: # Si está muerto
            if abs(row['y']-row[i][0]) < abs(row['y']-precio_proyectado_piso_muerto_mas_cercano) or np.isnan(precio_proyectado_piso_muerto_mas_cercano): # Si proyecta precio más cercano que el actual
              precio_proyectado_piso_muerto_mas_cercano = row[i][0]
              
          else: # Si está vivo
            if row[i][2] > nu_pruebas_piso_vivo_mas_probado or (np.isnan(nu_pruebas_piso_vivo_mas_probado) and row[i][2]>0): # Si fue más probado que el actual
              nu_pruebas_piso_vivo_mas_probado = row[i][2]
              precio_proyectado_piso_vivo_mas_probado = row[i][0]
              tendencia_piso_vivo_mas_probado = row[i][3]

            if (np.isnan(precio_proyectado_piso_vivo_mas_cercano)) or (abs(row['y']-row[i][0]) < abs(row['y']-precio_proyectado_piso_vivo_mas_cercano)): # Si, sin haber muerto, proyecta un techo más alto que el actual
              precio_proyectado_piso_vivo_mas_cercano = row[i][0]
        i = i + 1
        
    dataset.loc[index,'nu_pruebas_techo_vivo_mas_probado_'f"{lags}"] = nu_pruebas_techo_vivo_mas_probado
    dataset.loc[index,'precio_proyectado_techo_vivo_mas_probado_'f"{lags}"] = (precio_proyectado_techo_vivo_mas_probado - row['y'])/row['y']
    dataset.loc[index,'precio_proyectado_techo_vivo_mas_cercano_'f"{lags}"] = (precio_proyectado_techo_vivo_mas_cercano - row['y'])/row['y']
    dataset.loc[index,'precio_proyectado_techo_muerto_mas_cercano_'f"{lags}"] = (precio_proyectado_techo_muerto_mas_cercano - row['y'])/row['y']
    dataset.loc[index,'tendencia_techo_vivo_mas_probado_'f"{lags}"] = tendencia_techo_vivo_mas_probado/row['y']

    dataset.loc[index,'nu_pruebas_piso_vivo_mas_probado_'f"{lags}"] = nu_pruebas_piso_vivo_mas_probado
    dataset.loc[index,'precio_proyectado_piso_vivo_mas_probado_'f"{lags}"] = (precio_proyectado_piso_vivo_mas_probado - row['y'])/row['y']
    dataset.loc[index,'precio_proyectado_piso_vivo_mas_cercano_'f"{lags}"] = (precio_proyectado_piso_vivo_mas_cercano - row['y'])/row['y']
    dataset.loc[index,'precio_proyectado_piso_muerto_mas_cercano_'f"{lags}"] = (precio_proyectado_piso_muerto_mas_cercano - row['y'])/row['y']
    dataset.loc[index,'tendencia_piso_vivo_mas_probado_'f"{lags}"] = tendencia_piso_vivo_mas_probado/row['y']
    #print("Iteré sobre los precios y construí variables")

  return dataset

In [ ]:
today = date.today()
fc_empieza = today + timedelta(days=-5000)
fc_termina = today + timedelta(days=2)

mvl = yf.download('^MERV', start=fc_empieza, end=fc_termina)
print("Descargado Merval")
mvl = mvl[['Close']]
mvl.reset_index(level=0, inplace=True)
mvl.columns=['fc','mvl']
base = pd.DataFrame()

for ticker in (
    'GGAL.BA',
    'BMA.BA',
    'BYMA.BA',
    'CEPU.BA',
    'COME.BA',
    'CRES.BA',
    'CVH.BA',
    'EDN.BA',
    'MIRG.BA',
    'PAMP.BA',
    'SUPV.BA',
    'TECO2.BA',
    'TGNO4.BA',
    'TGSU2.BA',
    'TRAN.BA',
    'YPFD.BA'
):
  df = descarga(ticker, fc_empieza, fc_termina) # (Días empieza, días termina)
  print("Descargado ", ticker)
  
  df = calcula_pc_merval(df)
  df = calcula_amplitud(df)
  df = estandariza_volumen(df)
 
  df = calcula_medias(df)
  df = calcula_historia(df, 5) # (Lags)
  df = calcula_canalidad_y(df)
  df = calcula_canalidad_histog_macd(df)

  for per in (360, 120, 90, 60, 30, 15, 8, 4):
    df = calcula_AT_tendencias(df,per)
    print("Calculé AT para", ticker, "en lags de", per)
  
  df = df.tail(1)
  base = base.append(df)

In [6]:
mLink = 'https://github.com/fedefliguer/trading/blob/master/v1/v1_model.dat?raw=true'
mfile = BytesIO(requests.get(mLink).content)
model = pickle.load(mfile)
X=base.iloc[:, 6:]
preds = model.predict_proba(X)[:,1]
prediccions = base.iloc[:, 0:3]
prediccions['pred'] = preds
prediccions.sort_values(by=['pred'], ascending=False)

,fc,ticker,y,pred
3352,2021-02-01,TRAN.BA,26.700001,0.668461
3353,2021-02-01,PAMP.BA,79.500000,0.620433
3356,2021-02-01,CEPU.BA,35.000000,0.606689
3353,2021-02-01,EDN.BA,26.000000,0.599605
3353,2021-02-01,TGNO4.BA,42.250000,0.566848
3353,2021-02-01,COME.BA,2.440000,0.448574
3353,2021-02-01,CRES.BA,74.800003,0.392690
3353,2021-02-01,BMA.BA,209.000000,0.387761
3353,2021-02-01,TGSU2.BA,152.699997,0.384938
876,2021-02-01,BYMA.BA,602.000000,0.359412
